<a href="https://colab.research.google.com/github/anabossler/clasetecnolog-a/blob/master/_falta_revisar_ultima_parte_Ana_de_Souza_Bossler_Aprendizaje_Autom%C3%A1tico_Sesi%C3%B3n_2_Pr%C3%A1ctica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://drive.google.com/uc?id=1DHE8rHnhKRam8LrZpOkVY6iF3GMK7Jwf' caption="Máster Universitario en Automática y Robótica"></center>

# Aprendizaje Automático
## Práctica 2

Profesor: **José Javier Valero Mas**

### Ejercicio

La tarea a realizar consistirá en poner en práctica los conceptos prácticos introducidos en esta sesión.

Para ello el alumno deberá:
* Escoger un conjunto de datos desbalanceados para su clasificación
* Evaluar los cuatro clasificadores introducidos en la sesión teórica: $k$NN, Decision Tree, SVM y Random Forest
* Realizar una validación cruzada de k=10 folds
* Proporcionar el valor de la métrica F$_{1}$ para la clase **minoritaria** del conjunto de datos para los diferentes folds
* Promediar los 10 folds de cada clasificador y así concluir cuál se comporta mejor ante el escenario planteado


Para la realización de la misma se deberá utilizar alguno de estos dos conjuntos de datos:
* https://www.vision.uji.es/~sanchez/Databases/phoneme/dataset
* https://www.vision.uji.es/~sanchez/Databases/spam/dataset


### Entrega

  * Haz una copia de este archivo ("Archivo" > "Guardar una copia en drive").

  * Realiza la práctica descrita en el apartado anterior sobre el nuevo archivo. Documenta los pasos que vas realizando incrustando texto en Colab.

  * Genera un enlace para compartir (acuérdate de dar los permisos apropiados). Copia y pega el enlace en la entrega habilitada en el Moodle de la asignatura.



1. Elijo el dataset phoneme/dataset y analizo los datos brevemente. Vemos que los datos están desbalanceados, por ej. la columna 5 tiene 29,3% de los datos como 1 (son binarios, 1 o 0).



In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/sample_data/phoneme - tesisua.csv', header=None, delimiter=',')

X = data.loc[:,0:4]
Y = data.loc[:,5]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

In [56]:
print(data)


            0        1        2         3        4  5
0    -437.509  190.497 -185.775   -3.4440 -488.343  1
1    -443.875   53.265  132.394    1.6146    0.000  0
2    -504.454  420.224  922.828 -356.8210 -356.589  1
3    -515.856  651.163  120.677    1.2630    0.000  1
4    -532.765  432.019 -486.414 -243.4740 -161.961  0
...       ...      ...      ...       ...      ... ..
5399  380.153  -39.313  152.672 -824.4270 -549.618  0
5400  386.805  -33.112 -118.672 -655.6020 -439.834  0
5401  389.224 -241.037  488.353 -126.5950  -66.032  0
5402  390.863  583.756 -129.949 -873.0960  568.528  0
5403  410.667  606.667 -106.667 -466.6670 -333.333  0

[5404 rows x 6 columns]


In [57]:
data.head()

,0,1,2,3,4,5
0,-437.509,190.497,-185.775,-3.4440,-488.343,1
1,-443.875,53.265,132.394,1.6146,0.000,0
2,-504.454,420.224,922.828,-356.8210,-356.589,1
3,-515.856,651.163,120.677,1.2630,0.000,1
4,-532.765,432.019,-486.414,-243.4740,-161.961,0


In [58]:
print(f'The number of rows are {data.shape[0]} \nand the number of columns are {data.shape[1]}')

The number of rows are 5404 
and the number of columns are 6


In [59]:
print(data.dtypes)

0    float64
1    float64
2    float64
3    float64
4    float64
5      int64
dtype: object


In [60]:
data.describe()

,0,1,2,3,4,5
count,5404.000000,5404.000000,5404.000000,5404.000000,5404.000000,5404.000000
mean,309.876047,318.750186,147.126709,68.567513,-57.725222,0.293486
std,272.277477,283.813170,397.052486,461.037113,411.715776,0.455401
min,-993.032000,-917.279000,-997.305000,-999.329000,-997.151000,0.000000
25%,138.593500,132.113500,-42.288250,-211.998000,-297.444250,0.000000
50%,255.707000,239.312000,157.360000,121.996000,-4.161000,0.000000
75%,449.490500,514.506000,336.758750,320.282500,130.950250,1.000000
max,998.798000,999.631000,999.469000,999.773000,999.627000,1.000000


In [61]:
def missing(df):
    total=df.isnull().sum().sort_values(ascending=False)
    percent=(df.isnull().sum()*100/df.isnull().count()).sort_values(ascending=False)

In [62]:
missing(data)

In [63]:
data.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64

In [64]:
corrMatrix = data.corr()

In [22]:
print (corrMatrix)

          0         1         2         3         4         5
0  1.000000 -0.017077 -0.056228  0.033342  0.043353  0.030042
1 -0.017077  1.000000  0.140112  0.090821  0.038627  0.213518
2 -0.056228  0.140112  1.000000  0.094904 -0.015924  0.232598
3  0.033342  0.090821  0.094904  1.000000  0.069263  0.124206
4  0.043353  0.038627 -0.015924  0.069263  1.000000  0.051198
5  0.030042  0.213518  0.232598  0.124206  0.051198  1.000000


2.Aplicación de los 4 clasificadores.

In [65]:
#primero clasificador KNN 
from sklearn import neighbors

###se utiliza los parámetros default: distancia euclídea y k=5
model = neighbors.KNeighborsClassifier().fit(X_train, Y_train)

#predicción sobre la partición de test
Y_predict = model.predict(X_test)


In [66]:
from sklearn.metrics import accuracy_score
acc_sklearn = accuracy_score(Y_test, Y_predict)

print("Acc (sklearn): {}".format(acc_sklearn))

Acc (sklearn): 0.7971872686898593


En este caso tenemos un ACC de 0.792 por lo que habría acertado un 79,2% de las predicciones realizadas. 

Con un árbol de decisión:

In [67]:
from sklearn import tree

#árbol de decisión importado con parámetros default
model = tree.DecisionTreeClassifier().fit(X_train,Y_train)

Y_predict = model.predict(X_test)

print("Acc: {}".format(accuracy_score(Y_test, Y_predict)))

Acc: 0.768319763138416


In [ ]:
En este caso tenemos un ACC de 0.772.

Con una SVM:

In [68]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import svm


In [69]:
model = svm.SVC().fit(X_train,Y_train)

In [70]:
Y_predict = model.predict(X_test)
print("Acc: {}".format(accuracy_score(Y_test, Y_predict)))

Acc: 0.7379718726868986


Tenemos un ACC de 0.735


Con una random forest.




In [71]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier().fit(X_train,Y_train)
Y_predict = model.predict(X_test)
print("Acc: {}".format(accuracy_score(Y_test, Y_predict)))


Acc: 0.8393782383419689


Tenemos un ACC del 0.839.


Para continúar desbalanceamos más los datos:
no he podido indexar porque SVC no lo permitía

In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/sample_data/phoneme - tesisua.csv', header=None, delimiter=',')

X = data.loc[:,0:4]
Y = data.loc[:,5]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)


In [85]:
histogr = {i:list(Y).count(i) for i in set(Y)}
print("La distribución inicial de clases es: {}".format(histogr))

data_binary = pd.concat([data[data[0]==0][:100], data[data[0]!=0]])

X_binary = data_binary.loc[:,1:]
Y_binary = data_binary.loc[:,0]
Y_binary.iloc[100:]=1
Y_binary = Y_binary.astype('int')
histogr = {i:list(Y_binary).count(i) for i in set(Y_binary)}
print("La distribución binaria y desbalanceada es: {}".format(histogr))

La distribución inicial de clases es: {0: 3818, 1: 1586}
La distribución binaria y desbalanceada es: {1: 5304, -894: 1, -765: 1, -125: 3, -633: 1, -504: 1, -761: 1, -120: 1, -885: 1, -117: 1, -151: 1, -114: 3, -113: 1, -111: 2, -109: 1, -106: 2, -105: 2, -104: 4, -103: 3, -102: 3, -869: 1, -101: 1, -992: 1, -94: 1, -732: 1, -982: 1, -724: 1, -83: 1, -851: 1, -592: 1, -75: 1, -969: 1, -580: 1, -67: 1, -831: 1, -443: 1, -58: 1, -951: 1, -437: 1, -693: 1, -820: 1, -137: 1, -687: 1, -814: 1, -165: 2, -164: 3, -162: 1, -158: 1, -541: 1, -157: 1, -155: 1, -154: 1, -920: 1, -791: 1, -132: 2, -533: 1, -532: 1, -788: 1, -146: 3, -147: 1, -144: 1, -143: 3, -142: 1, -13: 1, -12: 2, -11: 3, -1: 1, -905: 1, -136: 1, -138: 1, -141: 1, -14: 2, -900: 1, -515: 1, -642: 1}


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [86]:
X_binary_train, X_binary_test, Y_binary_train, Y_binary_test = train_test_split(X_binary, Y_binary, test_size = 0.25)



In [98]:
model_binary = tree.DecisionTreeClassifier().fit(X_binary_train,Y_binary_train)

model_binary = neighbors.KNeighborsClassifier().fit(X_binary_train, Y_binary_train)

Y_binary_predict = model_binary.predict(X_test)

In [103]:
from sklearn.metrics import precision_score,recall_score,f1_score

print("Acc:\t{:.4f}".format(accuracy_score(Y_binary_test, Y_binary_predict)))
precision_score(Y_binary_test, Y_binary_predict, average='weighted')
precision_score(Y_binary_test, Y_binary_predict, average='micro')
print("-"*20)
print(precision_score)
print("Acc (all 1s):\t{:.4f}".format(accuracy_score(Y_binary_test, [1 for u in Y_binary_predict]))) 

Acc:	0.9830
--------------------
<function precision_score at 0x7f83a5c16048>
Acc (all 1s):	0.9830


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
#Importamos el método para la validación cruzada:
from sklearn.model_selection import cross_val_score

#Creamos un clasificador (árbol de decisión con parámetros por defecto):
model_CV = tree.DecisionTreeClassifier()

#Realizamos la validación cruzada con k=10 particiones:
scores = cross_val_score(model_CV, X_binary, Y_binary, cv=10)

#Mostramos los resultados por cada partición:
print(scores)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


[0.9038817  0.92051756 0.96118299 0.96857671 0.9537037  0.97222222
 0.97407407 0.97592593 0.97407407 0.91666667]


In [106]:
#Incluimos el método make_scorer, cambias la métrica porque suele usar sklearn accuracy pero maker score balancea las clases:
from sklearn.metrics import make_scorer

#Realizamos la misma validación cruzada con k=5 particiones pero usando la métrica F1 sobre la clase 0:
scores = cross_val_score(model_CV, X_binary, Y_binary, cv=10, scoring=make_scorer(f1_score, average='weighted'))

#Mostramos los resultados por cada partición:
print(scores)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


[0.93368356 0.94467345 0.9629264  0.96769155 0.96377194 0.9676578
 0.97225703 0.96952341 0.97043291 0.95434806]
